In [33]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import keras

In [34]:
classifier = Sequential()


In [35]:
classifier.add(Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)))

In [36]:
classifier.add(MaxPooling2D(pool_size = (2, 2)))


In [37]:
classifier.add(Conv2D(32, (3, 3), activation="relu"))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [38]:
classifier.add(Flatten())

In [39]:
classifier.add(Dense(activation = 'relu', units = 128))
classifier.add(Dense(1, activation='softmax'))

In [40]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [41]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [42]:
test_datagen = ImageDataGenerator(rescale = 1./255)


In [43]:
training_set = train_datagen.flow_from_directory('selectedData/image2class/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('selectedData/image2class/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 66 images belonging to 2 classes.
Found 33 images belonging to 2 classes.


In [44]:
history = classifier.fit_generator(training_set,
                         steps_per_epoch = 20,
                         epochs = 10,
                         validation_data = test_set,
                         validation_steps = 10)

Epoch 1/10
20/20 [==============================] - 23s 1s/step - loss: 7.2849 - acc: 0.5431 - val_loss: 7.2465 - val_acc: 0.5455
Epoch 2/10
20/20 [==============================] - 20s 1s/step - loss: 7.8692 - acc: 0.5064 - val_loss: 7.2465 - val_acc: 0.5455
Epoch 3/10
20/20 [==============================] - 20s 1s/step - loss: 8.4035 - acc: 0.4729 - val_loss: 7.2465 - val_acc: 0.5455
Epoch 4/10
20/20 [==============================] - 22s 1s/step - loss: 7.9458 - acc: 0.5016 - val_loss: 7.2465 - val_acc: 0.5455
Epoch 5/10
20/20 [==============================] - 20s 1s/step - loss: 8.0222 - acc: 0.4968 - val_loss: 7.2465 - val_acc: 0.5455
Epoch 6/10
20/20 [==============================] - 22s 1s/step - loss: 7.5898 - acc: 0.5239 - val_loss: 7.2465 - val_acc: 0.5455
Epoch 7/10
20/20 [==============================] - 26s 1s/step - loss: 7.6153 - acc: 0.5223 - val_loss: 7.2465 - val_acc: 0.5455
Epoch 8/10
20/20 [==============================] - 25s 1s/step - loss: 8.0222 - acc: 0.49

In [52]:
model = keras.models.Sequential()

In [53]:

# first Conv layer
model.add(keras.layers.Conv2D(
    16,
    (3,3),
    activation = 'relu',
    input_shape = (64, 64,3)
))
model.add(keras.layers.MaxPooling2D((2, 2)))

#second
model.add(keras.layers.Conv2D(
    32,
    (3,3),
    activation = 'relu'
))
model.add(keras.layers.MaxPooling2D((2, 2)))

#3rd
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation = 'relu'))
#4th
model.add(keras.layers.Dense(1, activation = 'softmax'))

adam_opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model.compile(optimizer= adam_opt, 
              loss='binary_crossentropy',
              metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 29, 29, 32)        4640      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                401472    
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 65        
Total para

In [54]:
## live plot for realtime performance visualization 
class LivePlot(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.fig = plt.figure()
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        
        clear_output(wait=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.losses, label="training loss")
        ax1.plot(self.x, self.val_losses, label="validation loss")
        ax1.legend()
        
        ax2.plot(self.x, self.acc, label="training accuracy")
        ax2.plot(self.x, self.val_acc, label="validation accuracy")
        ax2.legend()
        plt.show();      
lossPlot = LivePlot()

In [55]:
history = model.fit_generator(training_set,
                         steps_per_epoch = 100,
                         epochs = 10,
                         validation_data = test_set,
                         validation_steps = 10),
                         #callbacks = [lossPlot])

#predict the class labels
#score = model.evaluate(x_test, y_test, verbose = 0)
#print('Test accuracy: %.2f%%' % score[1])

print('end')

Epoch 1/10
100/100 [==============================] - 105s 1s/step - loss: 8.0352 - acc: 0.4960 - val_loss: 7.1499 - val_acc: 0.5515
Epoch 2/10
100/100 [==============================] - 105s 1s/step - loss: 7.5423 - acc: 0.5269 - val_loss: 7.3432 - val_acc: 0.5394
Epoch 3/10
100/100 [==============================] - 118s 1s/step - loss: 7.8182 - acc: 0.5096 - val_loss: 7.2465 - val_acc: 0.5455
Epoch 4/10
100/100 [==============================] - 142s 1s/step - loss: 7.7542 - acc: 0.5136 - val_loss: 7.1499 - val_acc: 0.5515
Epoch 5/10
100/100 [==============================] - 148s 1s/step - loss: 8.1932 - acc: 0.4861 - val_loss: 7.2465 - val_acc: 0.5455
Epoch 6/10
100/100 [==============================] - 148s 1s/step - loss: 8.2621 - acc: 0.4817 - val_loss: 7.3432 - val_acc: 0.5394
Epoch 7/10
100/100 [==============================] - 161s 2s/step - loss: 8.4941 - acc: 0.4672 - val_loss: 7.1499 - val_acc: 0.5515
Epoch 8/10
100/100 [==============================] - 156s 2s/step - 

In [56]:
#Confution Matrix and Classification Report
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict_generator(test_set,300 // 20)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
print('Classification Report')
target_names = ['Cats', 'Dogs', 'Horse']
print(classification_report(validation_generator.classes, y_pred))#, target_names=target_names))

Confusion Matrix


ValueError: Found input variables with inconsistent numbers of samples: [33, 232]

In [57]:
test_set.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [58]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)